In [40]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [41]:
from tfinterface.model_base import ModelBase
from tfinterface.reinforcement import ExperienceReplay
from tfinterface.utils import select_columns, soft_if, get_run
from phi.api import *
import tensorflow as tf
import random
from scipy.interpolate import interp1d
import numpy as np
import gym
from gym import wrappers
from tfinterface.reinforcement import ExpandedStateEnv
import os
import time


name = "actor-critic-base"

In [42]:
def huber_loss(x):
  return tf.where(
      tf.abs(x) < 1.0,
      0.5 * tf.square(x), 
      tf.abs(x) - 0.5
  )

def log_shift_loss(x, alfa=0.1):
    return - tf.log(x + alfa * (1.0 - x))

class Inputs(object):
    def __init__(self, n_states, scope):
        with tf.variable_scope(scope):
            self.episode_length = tf.placeholder(tf.int64, [], name='episode_length')

            self.s = tf.placeholder(tf.float32, [None, n_states], name='s')
            self.a = tf.placeholder(tf.int32, [None], name='a')
            self.r = tf.placeholder(tf.float32, [None], name='r')
            self.v1 = tf.placeholder(tf.float32, [None], name='V1')
            self.done = tf.placeholder(tf.float32, [None], name='done')
            
            self.learning_rate = tf.placeholder(tf.float32, [], name='learning_rate')
            self.keep_prob = tf.placeholder(tf.float32, [], name='keep_prob')
            self.training = tf.placeholder(tf.bool, [], name='training')
            
            self.pi = tf.placeholder(tf.float32, [], name='pi')
            

class Critic(object):
    def __init__(self, base_model, inputs, n_actions, n_states, y, scope):
        with tf.variable_scope(scope):
            
            ops = dict(
                trainable=True,
                kernel_initializer=tf.random_uniform_initializer(minval=0.0, maxval=0.01),
                bias_initializer=tf.random_uniform_initializer(minval=0.0, maxval=0.01)
            )

            net = inputs.s

            net = tf.layers.dense(net, 64, activation=tf.nn.relu, name="relu_layer", **ops)        
            
            self.V = tf.layers.dense(net, n_actions, name='V', **ops)[:, 0]

            self.target = soft_if(inputs.done, inputs.r,  inputs.r + y * inputs.v1)

            self.error = self.target - self.V
            self.loss = Pipe(self.error, tf.nn.l2_loss, tf.reduce_mean)

            self.variables = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES, scope=scope)

            self.update = tf.train.AdamOptimizer(inputs.learning_rate).minimize(self.loss, var_list=self.variables)

            avg_error, std_error = tf.nn.moments(self.error, [0])
            self.summaries = tf.summary.merge([
                tf.summary.scalar('loss', self.loss),
                tf.summary.scalar('avg_target', tf.reduce_mean(self.target)),
                tf.summary.scalar('variables_sum', sum([ tf.reduce_sum(v) for v in self.variables ])),
                tf.summary.scalar('avg_error', avg_error),
                tf.summary.scalar('std_error', std_error),
                tf.summary.histogram(
                    'avg_action', Pipe(
                    inputs.a,
                    Then(tf.one_hot, n_actions),
                    Then(tf.reduce_mean, axis=0)
                ))
            ]+[
                tf.summary.histogram('var{}'.format(i), self.variables[i]) for i in range(len(self.variables))
            ])
            
class Actor(object):
    def __init__(self, base_model, inputs, target_critic, n_actions, n_states, y, scope):
        with tf.variable_scope(scope):
            ops = dict(
                trainable=True,
                kernel_initializer=tf.random_uniform_initializer(minval=0.0, maxval=0.01),
                bias_initializer=tf.random_uniform_initializer(minval=0.0, maxval=0.01)
            )

            net = inputs.s

            net = tf.layers.dense(net, 64, activation=tf.nn.relu, name="relu_layer", use_bias=True, **ops)
            net = tf.nn.dropout(net, inputs.keep_prob)

            self.P = tf.layers.dense(net, n_actions, activation=tf.nn.softmax, name='P', use_bias=False, **ops)

            self.Pa = select_columns(self.P, inputs.a)

            self.loss = log_shift_loss(self.Pa) * target_critic.error
            self.loss = tf.reduce_mean(self.loss)

            self.variables = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES, scope=scope)

            self.update = tf.train.AdamOptimizer(inputs.learning_rate).minimize(self.loss, var_list=self.variables)

            self.summaries = tf.summary.merge([
                tf.summary.scalar('loss', self.loss),
                tf.summary.scalar('variables_sum', sum([ tf.reduce_sum(v) for v in self.variables ])),
                tf.summary.histogram(
                    'avg_action', Pipe(
                    inputs.a,
                    Then(tf.one_hot, n_actions),
                    Then(tf.reduce_mean, axis=0)
                ))
            ]+[
                tf.summary.histogram('var{}'.format(i), self.variables[i]) for i in range(len(self.variables))
            ])
            


In [43]:
class LunarLander(ModelBase):
    
    def define_model(self, n_actions, n_states, y=0.98, buffer_length=50000, pi=0.1):
        self.global_max = float('-inf')

        self.replay_buffer = ExperienceReplay(5, max_length=buffer_length)


        with self.graph.as_default(), tf.device("cpu:0"):

            self.inputs = Inputs(n_states, "inputs")

            self.critic = Critic(self, self.inputs, n_actions, n_states, y, "critic")
            self.target_critic = Critic(self, self.inputs, n_actions, n_states, y, "target_critic")
            self.actor = Actor(self, self.inputs, self.target_critic, n_actions, n_states, y, "actor")

            self.update = tf.group(self.critic.update, self.actor.update)

            self.episode_length_summary = tf.summary.scalar('episode_length', self.inputs.episode_length)

            self.summaries = tf.summary.merge([self.actor.summaries, self.critic.summaries, self.target_critic.summaries])

            self.update_target = tf.group(*[
                t.assign_add(pi * (a - t)) for t, a in zip(self.target_critic.variables, self.critic.variables)
            ])
    
    
    def predict_feed(self, S):
        return {
            self.inputs.s: S,
            self.inputs.keep_prob: 1.0,
            self.inputs.training: False
        }
    
    def predict(self, state, e = 0.0):
        predict_feed = self.predict_feed([state])
        actions = self.sess.run(self.actor.P, feed_dict=predict_feed)
        actions = actions[0]
        n = len(actions)

        if random.random() < e:
            return random.randint(0, n-1)
        else:
            return np.random.choice(n, p=actions)
    
    def fit_feed(self, S, A, R, V1, Done, learning_rate, keep_prob):
        return {
            self.inputs.s: S,
            self.inputs.a: A,
            self.inputs.r: R,
            self.inputs.v1: V1,
            self.inputs.done: Done,
            self.inputs.learning_rate: learning_rate,
            self.inputs.keep_prob: keep_prob,
            self.inputs.training: True
        }
    
    
    def fit(self, env, keep_prob=0.5, e=0.01, learning_rate=0.01, print_step=10, 
            update_target_step = 32, episodes=100000, max_episode_length=float('inf'), batch_size=32):
        
        r_total = 0.

        for episode in range(episodes):
            done = False
            ep_step = 0
            s = env.reset()
            episode_length = 0
            ep_reward = 0.
            
            while not done and ep_step <= max_episode_length:
                self.global_step += 1
                episode_length += 1
                ep_step += 1
                
                
                _learning_rate = learning_rate(self.global_step) if hasattr(learning_rate, '__call__') else learning_rate
                _e = e(self.global_step) if hasattr(e, '__call__') else e
                
                
                a = self.predict(s, e = _e)
                s1, r, done, info = env.step(a)
                r_total += r
                ep_reward += r
                
                
                self.replay_buffer.append(s, a, r, s1, float(done))
                
                
                S, A, R, S1, Done = self.replay_buffer.random_batch(batch_size).unzip()
                predict_feed = self.predict_feed(S1)
                V1 = self.sess.run(self.target_critic.V, feed_dict=predict_feed)

                
                fit_feed = self.fit_feed(S, A, R, V1, Done, _learning_rate, keep_prob)
                _, summaries = self.sess.run([self.update, self.summaries], feed_dict=fit_feed)
                self.writer.add_summary(summaries, self.global_step)
                
                
                if self.global_step % update_target_step == 0:
                    self.sess.run(self.update_target)
                
                
                s = s1
                
            
            episode_length_summary = self.sess.run(self.episode_length_summary,
                                                   feed_dict={self.inputs.episode_length: episode_length})
            self.writer.add_summary(episode_length_summary, self.global_step)


            if ep_reward >= self.global_max:
                print("[MAX] Episode: {}, Length: {}, Reward: {}, buffer_len: {}".format(episode, episode_length, ep_reward, len(self.replay_buffer)))
                self.save(model_path = self.model_path + ".{score}".format(score = ep_reward))
                self.global_max = ep_reward


            if episode % print_step == 0 and episode > 0:
                avg_r = r_total / print_step
                actor_loss = self.sess.run(self.actor.loss, feed_dict=fit_feed)
                print("[NOR] Episode: {}, Length: {}, Avg Reward: {}, e: {}, Learning Rate: {}, buffer_len: {}".format(episode, episode_length, avg_r, _e, _learning_rate, len(self.replay_buffer)))
                print("Loss: {}".format(actor_loss))
                self.save()
                r_total = 0.


In [44]:
run = get_run()
env = gym.make("LunarLander-v2")
env = wrappers.Monitor(env, "monitor/{run}".format(run = run))
env = ExpandedStateEnv(env, 3)
n_actions = env.action_space.n
n_states = env.observation_space.shape[0] * 3
model_path =  "{path}/models/{run}".format(path = os.getcwd(), run = run)
logs_path = "{path}/logs/{run}".format(path = os.getcwd(), run = run)


model = LunarLander(
    n_actions, n_states, y=0.9999, 
    buffer_length=500000,
    model_path = model_path,
    logs_path = logs_path,
    restore = False,
    pi = 0.005
)

[2017-03-22 15:12:15,341] Making new env: LunarLander-v2
[2017-03-22 15:12:15,344] Creating monitor directory monitor/7


False


In [ ]:
k = 40000.
model.fit(
    env, print_step=10, 
    episodes=int(1e5), max_episode_length=10000, batch_size=32,
    learning_rate = 0.01, # lambda t: 0.05 * k / (k + t)
    e = interp1d([0, 300000], [0.4, 0.05], fill_value=0.05, bounds_error=False),
    keep_prob = 0.5,
    update_target_step = 1
)

[2017-03-22 15:12:16,471] Starting new video recorder writing to /data/neura-lab/q-learning/notebooks/lunar-lander/actor-critic/base/monitor/7/openaigym.video.9.937.video000000.mp4
[2017-03-22 15:12:20,274] Starting new video recorder writing to /data/neura-lab/q-learning/notebooks/lunar-lander/actor-critic/base/monitor/7/openaigym.video.9.937.video000001.mp4


[MAX] Episode: 0, Length: 109, Reward: -318.429336077, buffer_len: 109
[MAX] Episode: 4, Length: 177, Reward: -216.753287611, buffer_len: 771
[MAX] Episode: 5, Length: 100, Reward: -190.671057424, buffer_len: 871


[2017-03-22 15:12:23,842] Starting new video recorder writing to /data/neura-lab/q-learning/notebooks/lunar-lander/actor-critic/base/monitor/7/openaigym.video.9.937.video000008.mp4


[MAX] Episode: 8, Length: 1000, Reward: 76.0189345362, buffer_len: 2148
[NOR] Episode: 10, Length: 1000, Avg Reward: -232.798901981, e: 0.396213, Learning Rate: 0.01, buffer_len: 3247
Loss: -1.47408568859
[NOR] Episode: 20, Length: 206, Avg Reward: -148.964016264, e: 0.3915965, Learning Rate: 0.01, buffer_len: 7204
Loss: 0.354218035936


[2017-03-22 15:12:56,504] Starting new video recorder writing to /data/neura-lab/q-learning/notebooks/lunar-lander/actor-critic/base/monitor/7/openaigym.video.9.937.video000027.mp4


...

[NOR] Episode: 6000, Length: 356, Avg Reward: 163.253858747, e: 0.05, Learning Rate: 0.01, buffer_len: 500000
Loss: 0.121156938374
[NOR] Episode: 6010, Length: 779, Avg Reward: 166.721411405, e: 0.05, Learning Rate: 0.01, buffer_len: 500000
Loss: -1.46970653534
[NOR] Episode: 6020, Length: 612, Avg Reward: 160.904827696, e: 0.05, Learning Rate: 0.01, buffer_len: 500000
Loss: -0.680952012539
[NOR] Episode: 6030, Length: 649, Avg Reward: 63.649142126, e: 0.05, Learning Rate: 0.01, buffer_len: 500000
Loss: 1.09636831284
[NOR] Episode: 6040, Length: 1000, Avg Reward: 80.8720473089, e: 0.05, Learning Rate: 0.01, buffer_len: 500000
Loss: -1.33526682854
[NOR] Episode: 6050, Length: 916, Avg Reward: 127.943612326, e: 0.05, Learning Rate: 0.01, buffer_len: 500000
Loss: -1.14234089851
[NOR] Episode: 6060, Length: 719, Avg Reward: 106.16471152, e: 0.05, Learning Rate: 0.01, buffer_len: 500000
Loss: 1.03257608414
[NOR] Episode: 6070, Length: 1000, Avg Reward: -42.9373936299, e: 0.05, Learning Rate

[2017-03-22 18:48:57,793] Starting new video recorder writing to /data/neura-lab/q-learning/notebooks/lunar-lander/actor-critic/base/monitor/7/openaigym.video.9.937.video007000.mp4


[NOR] Episode: 7000, Length: 136, Avg Reward: -38.5197143772, e: 0.05, Learning Rate: 0.01, buffer_len: 500000
Loss: 5.56186485291
[NOR] Episode: 7010, Length: 100, Avg Reward: -62.1346302559, e: 0.05, Learning Rate: 0.01, buffer_len: 500000
Loss: 2.86078429222
[NOR] Episode: 7020, Length: 138, Avg Reward: -57.1282662654, e: 0.05, Learning Rate: 0.01, buffer_len: 500000
Loss: 2.17297291756
[NOR] Episode: 7030, Length: 684, Avg Reward: -76.6906029278, e: 0.05, Learning Rate: 0.01, buffer_len: 500000
Loss: -6.05068778992
[NOR] Episode: 7040, Length: 95, Avg Reward: -49.6346982769, e: 0.05, Learning Rate: 0.01, buffer_len: 500000
Loss: -5.04913377762
[NOR] Episode: 7050, Length: 106, Avg Reward: -102.657341885, e: 0.05, Learning Rate: 0.01, buffer_len: 500000
Loss: -0.0972244441509
[NOR] Episode: 7060, Length: 99, Avg Reward: -100.958811372, e: 0.05, Learning Rate: 0.01, buffer_len: 500000
Loss: -5.00362014771
[NOR] Episode: 7070, Length: 115, Avg Reward: -125.543605912, e: 0.05, Learning

[2017-03-22 18:56:06,849] Starting new video recorder writing to /data/neura-lab/q-learning/notebooks/lunar-lander/actor-critic/base/monitor/7/openaigym.video.9.937.video008000.mp4


[NOR] Episode: 8000, Length: 165, Avg Reward: -25.2481756866, e: 0.05, Learning Rate: 0.01, buffer_len: 500000
Loss: 0.641100525856
[NOR] Episode: 8010, Length: 532, Avg Reward: -1.74041249118, e: 0.05, Learning Rate: 0.01, buffer_len: 500000
Loss: -2.39502716064
[NOR] Episode: 8020, Length: 543, Avg Reward: 37.6396880134, e: 0.05, Learning Rate: 0.01, buffer_len: 500000
Loss: -2.34110212326
[NOR] Episode: 8030, Length: 816, Avg Reward: -80.3668358782, e: 0.05, Learning Rate: 0.01, buffer_len: 500000
Loss: -3.37681078911
[NOR] Episode: 8040, Length: 1000, Avg Reward: -179.533008986, e: 0.05, Learning Rate: 0.01, buffer_len: 500000
Loss: -0.982254981995
[NOR] Episode: 8050, Length: 1000, Avg Reward: -173.585373943, e: 0.05, Learning Rate: 0.01, buffer_len: 500000
Loss: -4.50900316238
[NOR] Episode: 8060, Length: 362, Avg Reward: -131.736192664, e: 0.05, Learning Rate: 0.01, buffer_len: 500000
Loss: 2.61961531639
[NOR] Episode: 8070, Length: 244, Avg Reward: -85.4132801276, e: 0.05, Lear

In [6]:

env = gym.make("LunarLander-v2")
env = ExpandedStateEnv(env, 3)
n_actions = env.action_space.n
n_states = env.observation_space.shape[0] * 3
model_path =  "{path}/{name}".format(path = os.getcwd(), name = name)
logs_path = "{path}/logs/".format(path = os.getcwd(), name = name)


model_run = LunarLander(
    n_actions, n_states,
    model_path = model_path,
    flush_secs = 3.0,
    restore = True
)

for i in range(100):
    s = env.reset()
    done = False
    total = 0.
    ep = 0
    while not done and ep < 700:
        ep += 1
        a = model_run.predict(s, 0.0)
        s, r, done, info = env.step(a)
        total += r
        env.render()
        time.sleep(0.01)
    
    print(total)
    
env.render(close=True)


[2017-03-17 18:20:33,289] Making new env: LunarLander-v2


False
212.63485096
222.209029125
137.585769854
180.875673014
206.109792056
244.175226865
219.223174017
226.254220443
189.530083255
180.200875147
226.533856294
215.507410864
257.758179742


ArgumentError: argument 2: <type 'exceptions.TypeError'>: wrong type